[Reference](http://qiita.com/matsuou1/items/db6e8c48fcfd791dd876)

In [2]:
library(dplyr)

# Window関数で何ができるのか

* ランキング（タイ順位考慮あり、なし等含む）
* 前日比、前週比（前後のレコードとの比較等）
* 累積（累積和等）
* 移動平均（Windowサイズの指定、Windowの位置、重み等）

# ランキング関数

|    Function  |                               Description                               |
|:------------:|:-----------------------------------------------------------------------:|
| row_number   | 昇順にランキングを付ける。同じ値がある場合は、最初に来た方を優先        |
| min_rank     | 昇順にランキングを付ける。同じ値がある場合は、同じ順位を付ける。gapあり |
| dense_rank   | 昇順にランキングを付ける。同じ値がある場合は、同じ順位を付ける。gapなし |
| percent_rank | min_rankを0～1にリスケールしたもの                                      |
| cume_dist    | 累積割合。percent_rankの累積和ではない。                                |
| ntile        | n個の群に分割する                                                       |

In [3]:
data(USArrests)
d <- USArrests
d %>% head(5)

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


Ranking of Murder

In [4]:
d %>% 
   dplyr::mutate(State=rownames(.)) %>%  
   dplyr::mutate(row_number=row_number(Murder)) %>%
   dplyr::mutate(min_rank=min_rank(Murder)) %>%
   dplyr::mutate(max_rank=min_rank(desc(Murder))) %>%
   dplyr::mutate(dense_rank=dense_rank(Murder)) %>%
   dplyr::mutate(percent_rank=percent_rank(Murder)) %>%
   dplyr::mutate(cume_dist=cume_dist(Murder)) %>%
   dplyr::mutate(ntile=ntile(Murder, 3)) %>%
   dplyr::select(-Assault, -UrbanPop, -Rape) %>% 
   head(5)

Murder,State,row_number,min_rank,max_rank,dense_rank,percent_rank,cume_dist,ntile
13.2,Alabama,44,44,6,39,0.8775510,0.90,3
10.0,Alaska,35,35,16,30,0.6938776,0.70,3
8.1,Arizona,29,29,22,25,0.5714286,0.58,2
8.8,Arkansas,31,31,20,27,0.6122449,0.62,2
9.0,California,32,32,18,28,0.6326531,0.66,2


# Offsets

In [5]:
data(austres)
d <- austres
d <- data.frame(Y=as.matrix(d), date=time(d))
d %>% head(5)

Y,date
13067.3,1971.25
13130.5,1971.50
13198.4,1971.75
13254.2,1972.00
13303.7,1972.25


In [6]:
# １つづつ前後にずらす
d %>%
   dplyr::mutate(lead=lead(Y)) %>%
   dplyr::mutate(lag=lag(Y)) %>%
   head(3)

Y,date,lead,lag
13067.3,1971.25,13130.5,NA
13130.5,1971.50,13198.4,13067.3
13198.4,1971.75,13254.2,13130.5


In [7]:
# １つづつ前後にずらす
d %>%
   dplyr::mutate(lead=lead(Y)) %>%
   dplyr::mutate(lag=lag(Y)) %>%
   slice(c(1:3, n()-2, n()-1, n()))

Y,date,lead,lag
13067.3,1971.25,13130.5,NA
13130.5,1971.50,13198.4,13067.3
13198.4,1971.75,13254.2,13130.5
17568.7,1992.75,17627.1,17526.0
17627.1,1993.00,17661.5,17568.7
17661.5,1993.25,NA,17627.1


In [8]:
# 3つづつ前後にずらす
d %>%
   dplyr::mutate(lead=lead(Y, n=3)) %>%
   dplyr::mutate(lag=lag(Y, n=3)) %>%
   head(4)

Y,date,lead,lag
13067.3,1971.25,13254.2,NA
13130.5,1971.50,13303.7,NA
13198.4,1971.75,13353.9,NA
13254.2,1972.00,13409.3,13067.3


In [9]:
# 欠損値の埋め方
d %>%
   dplyr::mutate(lead=lead(Y, default=0)) %>%
   dplyr::mutate(lag=lag(Y, default=0)) %>%
   slice(c(1:3, n()-2, n()-1, n()))

Y,date,lead,lag
13067.3,1971.25,13130.5,0.0
13130.5,1971.50,13198.4,13067.3
13198.4,1971.75,13254.2,13130.5
17568.7,1992.75,17627.1,17526.0
17627.1,1993.00,17661.5,17568.7
17661.5,1993.25,0.0,17627.1


In [10]:
# group内でのoffset
data_frame(x = c(1 , 2, 3, 4, 5) , y=c("aa" ,"aa" , "aa" , "bb" , "bb")) %>%
   dplyr::group_by(y) %>%
   dplyr::mutate(lead=lead(x)) %>%
   dplyr::mutate(lag=lag(x))

x,y,lead,lag
1,aa,2,NA
2,aa,3,1
3,aa,NA,2
4,bb,5,NA
5,bb,NA,4


# 累積関数

In [11]:
data_frame(x = c(1,2,3,4,5)) %>%
   dplyr::mutate(cumsum=cumsum(x)) %>%
   dplyr::mutate(cummin=cummin(x)) %>%
   dplyr::mutate(cummax=cummax(x)) %>%
   dplyr::mutate(cummean=cummean(x))

x,cumsum,cummin,cummax,cummean
1,1,1,1,1.0
2,3,1,2,1.5
3,6,1,3,2.0
4,10,1,4,2.5
5,15,1,5,3.0


## そのほかの使い方は[元サイト](http://qiita.com/matsuou1/items/db6e8c48fcfd791dd876#%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E9%96%A2%E6%95%B0)を参考に